<a href="https://colab.research.google.com/github/rafabandoni/nfl-predict/blob/main/notebooks/00_nfl_predict_class_loading_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

# 00. Loading and Cleaning Data
Steps of this process:
1. Create one big raw dataset and split into train and test: this way we might have a real world scenario: data is gathered and then must be cleaned, but our test dataset might not be cleaned yet - as the to predict dataset won't arrive cleaned either.
2. Clean and document cleaning process on train dataset.
3. Create final dataset for EDA.

## Creating dataframes

In [3]:
PATH = 'https://raw.githubusercontent.com/rafabandoni/nfl-predict/refs/heads/main/data/input/'

In [4]:
score_historical = pd.read_csv(PATH + 'spreadspoke_scores.csv')
score_historical.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN


In [5]:
# Iter over a list so we don't need to call each df individually
data_list = [
  'defense_downs',
  'defense_fumbles',
  'defense_interceptions',
  'defense_passing',
  'defense_receiving',
  'defense_rushing',
  'defense_scoring',
  'defense_tackles',
  'offense_downs',
  'offense_passing',
  'offense_receiving',
  'offense_rushing',
  'offense_scoring',
  'special-teams_field-goals',
  'special-teams_kickoff-returns',
  'special-teams_kickoffs',
  'special-teams_punt-returns',
  'special-teams_punting',
  'special-teams_scoring',
]

In [6]:
dataframe_dict = {}
for item in data_list:
  data = pd.read_csv(PATH + f'{item}.csv')
  dataframe_dict[item] = data

In [7]:
# Unpack dict into each of the dataframe variables
(
  defense_downs,
  defense_fumbles,
  defense_interceptions,
  defense_passing,
  defense_receiving,
  defense_rushing,
  defense_scoring,
  defense_tackles,
  offense_downs,
  offense_passing,
  offense_receiving,
  offense_rushing,
  offense_scoring,
  special_teams_field_goals,
  special_teams_kickoff_returns,
  special_teams_kickoffs,
  special_teams_punt_returns,
  special_teams_punting,
  special_teams_scoring
) = tuple(dataframe_dict.values())

In [8]:
# Checking result
offense_rushing.head()

,Team,Att,Rush Yds,YPC,TD,20+,40+,Lng,Rush 1st,Rush 1st%,Rush FUM,year
0,Chiefs,556,2627,4.7,13,15,4,80T,129,23.2,7,2010
1,Jets,534,2374,4.4,14,11,2,53,118,22.1,11,2010
2,Jaguars,512,2395,4.7,14,14,1,74,148,28.9,8,2010
3,Raiders,504,2494,5.0,19,27,6,71,113,22.4,9,2010
4,Falcons,497,1891,3.8,14,12,1,55,111,22.3,4,2010


## Joining dataframes

So we can work with all as one.

In order to do better code and avoid repetitions, let's create a function!

**Note**: In class, let's first create the code and then the function

In [9]:
def get_df_name(df):
  name =[x for x in globals() if globals()[x] is df][0]
  return name

def change_dataframes_columns(dataframes_list):
  keep_names = ['year', 'Team']
  for df in dataframes_list:
    df_name = get_df_name(df)
    new_columns = []
    for column in df.columns:
      if not column in (keep_names):
        new_name = f'{df_name}_{column}'
      else:
        new_name = column
      new_columns.append(new_name)
    df.columns = new_columns

def merge_dataframes(dataframes_list):
  new_dataframe = dataframes_list[0].copy()
  for df in dataframes_list[1:]:
    new_dataframe = new_dataframe.merge(df,
                                        on=['year', 'Team'],
                                        how='left')
  return new_dataframe

In [10]:
defense_dfs = [
  defense_downs,
  defense_fumbles,
  defense_interceptions,
  defense_passing,
  defense_receiving,
  defense_rushing,
  defense_scoring,
  defense_tackles,
]

change_dataframes_columns(defense_dfs)
defense_dataframe = merge_dataframes(defense_dfs)

In [11]:
offense_dfs = [
  offense_downs,
  offense_passing,
  offense_receiving,
  offense_rushing,
  offense_scoring,
]

change_dataframes_columns(offense_dfs)
offense_dataframe = merge_dataframes(offense_dfs)

In [12]:
special_teams_dfs = [
  special_teams_field_goals,
  special_teams_kickoff_returns,
  special_teams_kickoffs,
  special_teams_punt_returns,
  special_teams_punting,
  special_teams_scoring
]

change_dataframes_columns(special_teams_dfs)
special_teams_dataframe = merge_dataframes(special_teams_dfs)

## Bringing all together as one

In [13]:
stats_dataframe = defense_dataframe.merge(offense_dataframe,
                                          on=['year', 'Team'],
                                          how='left')

stats_dataframe = stats_dataframe.merge(special_teams_dataframe,
                                        on=['year', 'Team'],
                                        how='left')

### Translating team names

In [14]:
set([*score_historical['team_home']] + [*score_historical['team_away']])

{'Arizona Cardinals',
 'Atlanta Falcons',
 'Baltimore Colts',
 'Baltimore Ravens',
 'Boston Patriots',
 'Buffalo Bills',
 'Carolina Panthers',
 'Chicago Bears',
 'Cincinnati Bengals',
 'Cleveland Browns',
 'Dallas Cowboys',
 'Denver Broncos',
 'Detroit Lions',
 'Green Bay Packers',
 'Houston Oilers',
 'Houston Texans',
 'Indianapolis Colts',
 'Jacksonville Jaguars',
 'Kansas City Chiefs',
 'Las Vegas Raiders',
 'Los Angeles Chargers',
 'Los Angeles Raiders',
 'Los Angeles Rams',
 'Miami Dolphins',
 'Minnesota Vikings',
 'New England Patriots',
 'New Orleans Saints',
 'New York Giants',
 'New York Jets',
 'Oakland Raiders',
 'Philadelphia Eagles',
 'Phoenix Cardinals',
 'Pittsburgh Steelers',
 'San Diego Chargers',
 'San Francisco 49ers',
 'Seattle Seahawks',
 'St. Louis Cardinals',
 'St. Louis Rams',
 'Tampa Bay Buccaneers',
 'Tennessee Oilers',
 'Tennessee Titans',
 'Washington Commanders',
 'Washington Football Team',
 'Washington Redskins'}

In [15]:
stats_dataframe['Team'].sort_values().unique()

array(['49ers', 'Bears', 'Bengals', 'Bills', 'Broncos', 'Browns',
       'Buccaneers', 'Cardinals', 'Chargers', 'Chiefs', 'Colts',
       'Commanders', 'Cowboys', 'Dolphins', 'Eagles', 'Falcons',
       'FootballTeam', 'Giants', 'Jaguars', 'Jets', 'Lions', 'Niners',
       'Packers', 'Panthers', 'Patriots', 'Raiders', 'Rams', 'Ravens',
       'Redskins', 'Saints', 'Seahawks', 'Steelers', 'Texans', 'Titans',
       'Vikings'], dtype=object)

In [16]:
teams_to_translate = {
    'Niners' : '49ers',
    'Redskins' : 'Commanders',
    'FootballTeam' : 'Commanders'
}

def translate_team_names(team_name, teams_to_translate):
  if team_name in teams_to_translate.keys():
    return teams_to_translate.get(team_name)
  else:
    return team_name

stats_dataframe['Team'] = stats_dataframe.apply(lambda row: translate_team_names(row['Team'], teams_to_translate), axis=1)

In [17]:
# Change Commanders name to get historic
commanders_name = [
    'Washington Commanders',
    'Washington Football Team',
    'Washington Redskins'
]

def change_commanders_name(team_name, commanders_name):
  if team_name in commanders_name:
    return 'Washington Commanders'
  else:
    return team_name

score_historical['team_home'] = score_historical.apply(lambda row: change_commanders_name(row['team_home'], commanders_name), axis=1)
score_historical['team_away'] = score_historical.apply(lambda row: change_commanders_name(row['team_away'], commanders_name), axis=1)

In [18]:
score_historical['team_home'] = score_historical['team_home'].str.split(' ').str[-1]
score_historical['team_away'] = score_historical['team_away'].str.split(' ').str[-1]

### Creating unique dataset

In [19]:
stats_dataframe['year'] = stats_dataframe['year'] + 1 # the last year stats refers to this years game
stats_dataframe.rename(columns={'year' : 'stats_year'}, inplace=True)

In [20]:
final_df = score_historical.merge(stats_dataframe,
                                  left_on=['schedule_season', 'team_home'],
                                  right_on=['stats_year', 'Team'],
                                  how='left').merge(stats_dataframe,
                                                    left_on=['schedule_season', 'team_away'],
                                                    right_on=['stats_year', 'Team'],
                                                    how='left',
                                                    suffixes=('_home_', '_away_'))

In [21]:
final_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,Team_home_,defense_downs_3rd Att_home_,defense_downs_3rd Md_home_,defense_downs_4th Att_home_,defense_downs_4th Md_home_,defense_downs_Rec 1st_home_,defense_downs_Rec 1st%_home_,defense_downs_Rush 1st_home_,defense_downs_Rush 1st%_home_,defense_downs_Scrm Plys_home_,stats_year_home_,defense_fumbles_FF_home_,defense_fumbles_FR_home_,defense_fumbles_FR TD_home_,defense_fumbles_Rec FUM_home_,defense_fumbles_Rush FUM_home_,defense_interceptions_INT_home_,defense_interceptions_INT TD_home_,defense_interceptions_INT Yds_home_,defense_interceptions_Lng_home_,defense_passing_Att_home_,defense_passing_Cmp_home_,defense_passing_Cmp %_home_,defense_passing_Yds/Att_home_,defense_passing_Yds_home_,defense_passing_TD_home_,defense_passing_INT_home_,defense_passing_Rate_home_,defense_passing_1st_home_,defense_passing_1st%_home_,defense_passing_20+_home_,defense_passing_40+_home_,defense_passing_Lng_home_,defense_passing_Sck_home_,defense_receiving_Rec_home_,defense_receiving_Yds_home_,defense_receiving_Yds/Rec_home_,defense_receiving_TD_home_,defense_receiving_20+_home_,defense_receiving_40+_home_,defense_receiving_Lng_home_,defense_receiving_Rec 1st_home_,defense_receiving_Rec 1st%_home_,defense_receiving_Rec FUM_home_,defense_receiving_PDef_home_,defense_rushing_Att_home_,defense_rushing_Rush Yds_home_,defense_rushing_YPC_home_,defense_rushing_TD_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_Lng_home_,defense_rushing_Rush 1st_home_,defense_rushing_Rush 1st%_home_,defense_rushing_Rush FUM_home_,defense_scoring_FR TD_home_,defense_scoring_SFTY_home_,defense_scoring_INT TD_home_,defense_tackles_Sck_home_,defense_tackles_Comb_home_,defense_tackles_Asst_home_,defense_tackles_Solo_home_,offense_downs_3rd Att_home_,offense_downs_3rd Md_home_,offense_downs_4th Att_home_,offense_downs_4th Md_home_,offense_downs_Rec 1st_home_,offense_downs_Rec 1st%_home_,offense_downs_Rush 1st_home_,offense_downs_Rush 1st%_home_,offense_downs_Scrm Plys_home_,offense_passing_Att_home_,offense_passing_Cmp_home_,offense_passing_Cmp %_home_,offense_passing_Yds/Att_home_,offense_passing_Pass Yds_home_,offense_passing_TD_home_,offense_passing_INT_home_,offense_passing_Rate_home_,offense_passing_1st_home_,offense_passing_1st%_home_,offense_passing_20+_home_,offense_passing_40+_home_,offense_passing_Lng_home_,offense_passing_Sck_home_,offense_passing_SckY_home_,offense_receiving_Rec_home_,offense_receiving_Yds_home_,offense_receiving_Yds/Rec_home_,offense_receiving_TD_home_,offense_receiving_20+_home_,offense_receiving_40+_home_,offense_receiving_Lng_home_,offense_receiving_Rec 1st_home_,offense_receiving_Rec 1st%_home_,offense_receiving_Rec FUM_home_,offense_rushing_Att_home_,offense_rushing_Rush Yds_home_,offense_rushing_YPC_home_,offense_rushing_TD_home_,offense_rushing_20+_home_,offense_rushing_40+_home_,offense_rushing_Lng_home_,offense_rushing_Rush 1st_home_,offense_rushing_Rush 1st%_home_,offense_rushing_Rush FUM_home_,offense_scoring_Rsh TD_home_,offense_scoring_Rec TD_home_,offense_scoring_Tot TD_home_,offense_scoring_2-PT_home_,special_teams_field_goals_FGM_home_,special_teams_field_goals_Att_home_,special_teams_field_goals_FG %_home_,special_teams_field_goals_1-19 > A-M_home_,special_teams_field_goals_20-29 > A-M_home_,special_teams_field_goals_30-39 > A-M_home_,special_teams_field_goals_40-49 > A-M_home_,special_teams_field_goals_50-59 > A-M_home_,special_teams_field_goals_60+ > A-M_home_,special_teams_field_goals_Lng_home_,special_teams_field_goals_FG Blk_home_,special_teams_kickoff_returns_Avg_home_,special_teams_kickoff_returns_Ret_home_,special_teams_kickoff_returns_Yds_home_,special_teams_kickoff_returns_KRet TD_home_,special_teams_kickoff_returns_20+_home_,special_teams_kickoff_returns_4

### Spliting data to be used as test
Let's create the test dataset, which should reflect real world and avoid data leackage.

In [22]:
final_df['schedule_season'].unique()

array([1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976,
       1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987,
       1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998,
       1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023, 2024])

In [23]:
train_data = final_df[final_df['schedule_season'] < 2024]
test_data = final_df[final_df['schedule_season'] == 2024]

From now on, we might work only with train data. We do this to work our train data as close as possible with the real world.  
When we get our data to predict, we might only translate the names if needed (as done above), and then we need to take care of all of the next processes, so we might create a **pipeline** after all to take care of it also in the test data.

## Cleaning data

## Cleaning data

Now that we already have 3 main datasets, we can start cleaning data! Cleaning data goes through some process, as such:
1. Check null values
2. Drop unused columns
3. Rename columns and/or values if needed
4. Check overall data

In [24]:
# removing data without stats
train_data = train_data[~train_data['Team_home_'].isna()]

### Creating functions

In [25]:
# Cleaning % data
def clean_percent_data(df):
  for column in df.columns:
    if '%' in column:
      df[column] = df[column] / 100

In [26]:
# Remove special characters from columns
def remove_special_char_columns(df):
  for column in df.columns:
    new_name = column.lower().replace(' ','_').replace('%', '_perc')
    df.rename(columns={
        column : new_name
    }, inplace=True)

In [27]:
# Transform turnover in negative data
def negative_turnover_number(turnover_columns, df):
  for column in turnover_columns:
    new_column = []
    for item in df[column]:
      if 'T' in item:
        new_value = item.replace('T', '')
        new_value = int(new_value)
        new_value = new_value * -1
      else:
        new_value = int(item)
      new_column.append(new_value)
    df[column] = new_column

In [28]:
def creating_home_and_away_columns(columns: list) -> list:
  new_named_list = []
  for item in columns:
    new_named_list.append(item + '_home_')
    new_named_list.append(item + '_away_')
  return new_named_list

### Applying functions

In [29]:
turnover_columns = [
    'defense_passing_lng',
    'defense_receiving_lng',
    'defense_rushing_lng',
    'defense_interceptions_lng',
    'offense_passing_lng',
    'offense_receiving_lng',
    'offense_rushing_lng',
    'special_teams_kickoff_returns_lng',
    'special_teams_punt_returns_lng',
    'special_teams_punting_lng'
]

turnover_columns = creating_home_and_away_columns(turnover_columns)

In [30]:
clean_percent_data(train_data)
remove_special_char_columns(train_data)
negative_turnover_number(turnover_columns, train_data)

In [31]:
# Fix columns with A_M (attemps_made)
columns_list = [
    'special_teams_field_goals_1-19_>_a-m',
    'special_teams_field_goals_20-29_>_a-m',
    'special_teams_field_goals_30-39_>_a-m',
    'special_teams_field_goals_40-49_>_a-m',
    'special_teams_field_goals_50-59_>_a-m',
    'special_teams_field_goals_60+_>_a-m',
]
columns_list = creating_home_and_away_columns(columns_list)

# We will transform it in a percent so we don't need to create a new column for each case
for column in columns_list:
  attps_list = train_data[column].str.split('_').str[0]
  matches_list = train_data[column].str.split('_').str[1]

  percent_matches = matches_list.astype('int') / attps_list.astype('int')
  percent_matches.fillna(0, inplace=True)
  train_data[column] = percent_matches
  train_data.rename(columns={
      column : column.replace('a-m', 'percent_a_m')
  }, inplace=True)

In [32]:
# Removing non important columns (trash from the origin)
columns_to_drop = ['data_fgm',
                   'data_fg__perc',
                   'data_xpm',
                   'data_xp_pct',
                   'data_kret_td',
                   'data_pret_t']
columns_to_drop = creating_home_and_away_columns(columns_to_drop)

train_data.drop(columns_to_drop,
                axis=1,
                inplace=True)

### Fixing data types

In [33]:
train_data['schedule_date'] = train_data['schedule_date'].astype('datetime64[ns]')

## Creating final df for next steps (EDA and modeling)

In [34]:
# create our Y, the data we want to predict
def home_winner(score_home, score_away):
  if score_home > score_away:
    return True
  else:
    return False # we will not be working on ties here since they are rare on NFL

train_data['home_winner'] = train_data.apply(lambda row: home_winner(row['score_home'], row['score_away']), axis=1)
test_data['home_winner'] = test_data.apply(lambda row: home_winner(row['score_home'], row['score_away']), axis=1)

<ipython-input-34-2d7d0add57d0>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data['home_winner'] = train_data.apply(lambda row: home_winner(row['score_home'], row['score_away']), axis=1)
<ipython-input-34-2d7d0add57d0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['home_winner'] = test_data.apply(lambda row: home_winner(row['score_home'], row['score_away']), axis=1)


In [35]:
# droping unused data
train_data.drop(['weather_temperature',
                 'weather_wind_mph',
                 'weather_humidity',
                 'weather_detail'],
                axis=1,
                inplace=True)

In [36]:
# Checking if any null value was left behind
train_data.isna().any().sort_values()

,0
schedule_date,False
defense_scoring_sfty_away_,False
defense_scoring_fr_td_away_,False
defense_rushing_rush_fum_away_,False
defense_rushing_rush_1st_perc_away_,False
...,...
offense_rushing_td_home_,False
offense_rushing_ypc_home_,False
offense_rushing_rush_yds_home_,False
offense_rushing_rush_fum_home_,False


This is the DF where we will make our EDA, feature engineering, and modeling :)

In [37]:
train_data.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,team_home_,defense_downs_3rd_att_home_,defense_downs_3rd_md_home_,defense_downs_4th_att_home_,defense_downs_4th_md_home_,defense_downs_rec_1st_home_,defense_downs_rec_1st_perc_home_,defense_downs_rush_1st_home_,defense_downs_rush_1st_perc_home_,defense_downs_scrm_plys_home_,stats_year_home_,defense_fumbles_ff_home_,defense_fumbles_fr_home_,defense_fumbles_fr_td_home_,defense_fumbles_rec_fum_home_,defense_fumbles_rush_fum_home_,defense_interceptions_int_home_,defense_interceptions_int_td_home_,defense_interceptions_int_yds_home_,defense_interceptions_lng_home_,defense_passing_att_home_,defense_passing_cmp_home_,defense_passing_cmp__perc_home_,defense_passing_yds/att_home_,defense_passing_yds_home_,defense_passing_td_home_,defense_passing_int_home_,defense_passing_rate_home_,defense_passing_1st_home_,defense_passing_1st_perc_home_,defense_passing_20+_home_,defense_passing_40+_home_,defense_passing_lng_home_,defense_passing_sck_home_,defense_receiving_rec_home_,defense_receiving_yds_home_,defense_receiving_yds/rec_home_,defense_receiving_td_home_,defense_receiving_20+_home_,defense_receiving_40+_home_,defense_receiving_lng_home_,defense_receiving_rec_1st_home_,defense_receiving_rec_1st_perc_home_,defense_receiving_rec_fum_home_,defense_receiving_pdef_home_,defense_rushing_att_home_,defense_rushing_rush_yds_home_,defense_rushing_ypc_home_,defense_rushing_td_home_,defense_rushing_20+_home_,defense_rushing_40+_home_,defense_rushing_lng_home_,defense_rushing_rush_1st_home_,defense_rushing_rush_1st_perc_home_,defense_rushing_rush_fum_home_,defense_scoring_fr_td_home_,defense_scoring_sfty_home_,defense_scoring_int_td_home_,defense_tackles_sck_home_,defense_tackles_comb_home_,defense_tackles_asst_home_,defense_tackles_solo_home_,offense_downs_3rd_att_home_,offense_downs_3rd_md_home_,offense_downs_4th_att_home_,offense_downs_4th_md_home_,offense_downs_rec_1st_home_,offense_downs_rec_1st_perc_home_,offense_downs_rush_1st_home_,offense_downs_rush_1st_perc_home_,offense_downs_scrm_plys_home_,offense_passing_att_home_,offense_passing_cmp_home_,offense_passing_cmp__perc_home_,offense_passing_yds/att_home_,offense_passing_pass_yds_home_,offense_passing_td_home_,offense_passing_int_home_,offense_passing_rate_home_,offense_passing_1st_home_,offense_passing_1st_perc_home_,offense_passing_20+_home_,offense_passing_40+_home_,offense_passing_lng_home_,offense_passing_sck_home_,offense_passing_scky_home_,offense_receiving_rec_home_,offense_receiving_yds_home_,offense_receiving_yds/rec_home_,offense_receiving_td_home_,offense_receiving_20+_home_,offense_receiving_40+_home_,offense_receiving_lng_home_,offense_receiving_rec_1st_home_,offense_receiving_rec_1st_perc_home_,offense_receiving_rec_fum_home_,offense_rushing_att_home_,offense_rushing_rush_yds_home_,offense_rushing_ypc_home_,offense_rushing_td_home_,offense_rushing_20+_home_,offense_rushing_40+_home_,offense_rushing_lng_home_,offense_rushing_rush_1st_home_,offense_rushing_rush_1st_perc_home_,offense_rushing_rush_fum_home_,offense_scoring_rsh_td_home_,offense_scoring_rec_td_home_,offense_scoring_tot_td_home_,offense_scoring_2-pt_home_,special_teams_field_goals_fgm_home_,special_teams_field_goals_att_home_,special_teams_field_goals_fg__perc_home_,special_teams_field_goals_1-19_>_percent_a_m_home_,special_teams_field_goals_20-29_>_percent_a_m_home_,special_teams_field_goals_30-39_>_percent_a_m_home_,special_teams_field_goals_40-49_>_percent_a_m_home_,special_teams_field_goals_50-59_>_percent_a_m_home_,special_teams_field_goals_60+_>_percent_a_m_home_,special_teams_field_goals_lng_home_,special_teams_field_goals_fg_blk_home_,special_teams_kickoff_returns_avg_home_,special_teams_kickoff_returns_ret_home_,special_teams_kickoff_returns_yds_home_,special_teams_kickoff_returns_kret_td_home_,special_teams_kickoff_returns_20+_home_,

## Saving the DF as parquet (a lighter format that preserves the fields).

In [38]:
train_data.to_parquet('train_data.parquet')
test_data.to_parquet('test_data.parquet')